# Re Fine-Tuning Model with Adjusted Tokenizer

This notebook contains a reimplementation of the fine tuning of distilbert for NER with the adjustment of a tokenizer that has been trained on both the conll2003 and wikipedia toxic comments corpus. This will benefit the labeling job because have the tokenizer's vocabulary contain words which it will be used to label will curb the tokenizer from subword tokenizing named toxic entities and disturbing the structural integrety of those statements for NER. The training script train.py contains these modifications. This notebook contains only the setup of a sagemaker env for that train.py and the training logs.

## Understanding this Implementation

For a more detailed explanation of how the model, task, and training metric works, visit: ./NER_standard_model/V1_model_base_tokenizer/sagemaker_training_env.ipynb

The tokenizer was trained in ./NER_standard_model/tokenizer_training/labeler_tokenizer_training.ipynb

The tokenizer vocab file is at ./NER_standard_model/tokenizer_training/tokenizer_files/conll_wiki_tokenizer-vocab.txt


# Sagemaker Env


In [2]:
import sagemaker
sess = sagemaker.Session() #this creates a sagemaker session -
role = sagemaker.get_execution_role() #this gets permissions from the env where 
                                      #it is running. I am running in a sagemaker notebook instance

# HuggingFace Estimator
##### The huggingface estimator is a tool that will create a docker image of our specified hyperparams and conduct the training specified within the train.py training script.

In [6]:
from sagemaker.huggingface import HuggingFace


# hyperparameters which are passed to the training job
hyperparameters={'epochs': 3,
                 'per_device_train_batch_size': 16,
                 'per_device_eval_batch_size': 16
                 }

# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='/home/ec2-user/SageMaker/NER_training_sagemaker/wiki-addition-training', #This is the only change made to the doc
        instance_type='ml.g4dn.xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.28.1',
        pytorch_version='2.0.0',
        py_version='py310',
        hyperparameters = hyperparameters
)

In [7]:
huggingface_estimator.fit({
    'train': "s3://conll2003-task/tokenized_conll2003/train/", 
    'test': "s3://conll2003-task/tokenized_conll2003/validation/"
    })

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2023-09-02-01-42-29-022


2023-09-02 01:42:29 Starting - Starting the training job.........
2023-09-02 01:43:51 Starting - Preparing the instances for training......
2023-09-02 01:44:52 Downloading - Downloading input data...
2023-09-02 01:45:13 Training - Downloading the training image.............................................
2023-09-02 01:52:50 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-09-02 01:53:02,136 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-09-02 01:53:02,153 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-09-02 01:53:02,163 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-09-02 01:53:02,169 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-09-02 01:53:04,229 sagemaker-training-toolkit 

Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=999265168ddfe7abd91119b8cff634b3e100bee02cff1613d076d2d2d87324a3
Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
/opt/ml/code/train.py:71: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval") #load in seqeval metric after install
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1713: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.

25%|██▍       | 328/1317 [00:43<02:06,  7.84it/s]


...

60%|█████▉    | 786/1317 [01:54<01:10,  7.57it/s]


98%|█████████▊| 50/51 [00:04<00:00,  9.15it/s]#033[A
#033[A
{'eval_loss': 0.2935323417186737, 'eval_precision': 0.6621699749470032, 'eval_recall': 0.616544051677732, 'eval_f1': 0.6385430217431703, 'eval_accuracy': 0.9107700005886855, 'eval_runtime': 6.5206, 'eval_samples_per_second': 498.423, 'eval_steps_per_second': 7.821, 'epoch': 2.0}
#015100%|██████████| 51/51 [00:06<00:00,  9.15it/s]#033[A
#015                                               #033[A
75%|███████▌  | 991/1317 [02:29<00:45,  7.19it/s]


76%|███████▌  | 1000/1317 [02:30<00:46,  6.83it/s]
{'loss': 0.3056, 'learning_rate': 1.9400244798041617e-05, 'epoch': 2.28}
91%|█████████ | 1192/1317 [02:59<00:17,  7.17it/s]


98%|█████████▊| 50/51 [00:05<00:00,  8.62it/s]#033[A
#033[A
{'eval_loss': 0.24818406999111176, 'eval_precision': 0.700082019502415, 'eval_recall': 0.6892158621927149, 'eval_f1': 0.6946064469460645, 'eval_accuracy': 0.9254135515394125, 'eval_runtime': 6.7961, 'eval_samples_per_second': 478.219, 'eval_steps_per_second': 7.504, 'epoch': 3.0}
#015100%|██████████| 51/51 [00:06<00:00,  8.62it/s]#033[A
#033[A
{'train_runtime': 204.3563, 'train_samples_per_second': 206.125, 'train_steps_per_second': 6.445, 'train_loss': 0.46218288424527365, 'epoch': 3.0}
90%|█████████ | 46/51 [00:04<00:00,  8.42it/s]


100%|██████████| 51/51 [00:06<00:00,  7.46it/s]
***** Eval results *****
2023-09-02 01:56:48,732 sagemaker-training-toolkit INFO     Waiting for the process to finish and give a return code.
2023-09-02 01:56:48,732 sagemaker-training-toolkit INFO     Done waiting for a return code. Received 0 from exiting process.
2023-09-02 01:56:48,732 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2023-09-02 01:56:51 Uploading - Uploading generated training model
2023-09-02 01:58:37 Completed - Training job completed
Training seconds: 824
Billable seconds: 824
